In [1]:
import yaml
import pandas as pd

cfg = yaml.load(open('configs/ukbb_test_20.yaml', "r"), Loader=yaml.Loader)

In [2]:
df = pd.read_csv(cfg['results_path'])
print(df['patient_id'].nunique())
print(len(df))

20
416


In [3]:
print("DICE Mean: ", df['dice_mean'].mean())
print("RV Mean: ", df['dice_rv'].mean())
print("MYO Mean: ", df['dice_myo'].mean())
print("LV Mean: ", df['dice_lv'].mean())

DICE Mean:  81.58120456243978
RV Mean:  78.05129358002513
MYO Mean:  79.65056939082933
LV Mean:  87.04175071646486


In [4]:
filtered_rows = df[df['dice_mean'] < 50]
print(len(filtered_rows))

79


In [5]:
filtered_rows['slice_idx'].value_counts()

slice_idx
8     15
9     12
1     10
10     8
0      8
7      8
2      7
11     6
3      2
12     1
6      1
4      1
Name: count, dtype: int64